In [2]:
import os
import numpy as np

from typing import (
    List,
    Tuple,
    Optional,
    Union
)

In [23]:
def generate_bvals(num_frames: int,
                   out_file: str = 'file.bval'
                  ) -> str:
    """Creates/generates an arbitrary number b=0 b-values for a fMRI acquisition
    provided the number of dynamics/temporal frames.
    
    Usage example:
        >>> bval_file = generate_bvals(num_frames=1200, out_file="fmri.bval")
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        out_file: Output file name.
        
    Returns:
        Output file path as string that contains the corresponding b-values.
    
    Raises:
        TypeError: Exception that is raised when the input for ``num_frames`` is not an ``int``.
    """
    if type(num_frames) != int:
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer.")
        
    np.savetxt(out_file,
               np.zeros((1,num_frames),
                        dtype=np.int8,
                        order='C'),
               fmt='%i',
               delimiter=' ',
               encoding='utf-8')
    out_file: str = os.path.abspath(out_file)
    return out_file

In [25]:
def generate_bvecs(num_frames: int,
                   out_file: str = 'file.bvec'
                  ) -> str:
    """Creates/generates an arbitrary number of x,y,z b-vectors for a fMRI acquisition
    provided the number of dynamics/temporal frames.
    
    Usage example:
        >>> bval_file = generate_bvecs(num_frames=1200, out_file="fmri.bvec")
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        out_file: Output file name.
        
    Returns:
        Output file path as string that contains the corresponding b-vectors.
    
    Raises:
        TypeError: Exception that is raised when the input for ``num_frames`` is not an ``int``.
    """
    if type(num_frames) != int:
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer.")
        
    np.savetxt(out_file,
               np.ones((1,num_frames),dtype=np.int8,order='C') * np.array([[1], [0], [0]]),
               fmt='%i',
               delimiter=' ',
               encoding='utf-8')
    out_file: str = os.path.abspath(out_file)
    return out_file

In [26]:
def generate_acq_params(num_frames: int,
                        effective_echo_spacing: float = 0.05,
                        out_prefix: str = 'file'
                       ) -> Tuple[str,str]:
    """Creates acquisition parameters files for fMRI acquisition provided the number of frames and the 
    effective echo spacing.
    
    Usage example:
        >>> dist_acqp, func_acqp = generate_acq_params(num_frames=1200, 
        ...                                            effective_echo_spacing=0.05,
        ...                                            out_prefix="fmri")
        ...
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        effective_echo_spacing: Effective echo spacing.
        out_prefix: Output file prefix.
        
    Returns:
        Tuple of strings that correspond to the acquisition parameter files (for both distortion correction and
            eddy current correction).
    
    Raises:
        TypeError: Exception that is raised when either ``num_frames`` is not an ``int`` OR when ``effective_echo_spacing`` is not a ``float``.
    """
    if (type(num_frames) != int) or (type(effective_echo_spacing) != float):
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer OR effective_echo_spacing: {effective_echo_spacing} is not a float.")
    
    # Generate distortion correction
    out_dist: str = out_prefix + "_distortion_correction.acqp"
    out_func: str = out_prefix + "_functional.acqp"
    
    # Write distortion correction acqp file
    with open(out_dist,'w') as f:
        f.write(f"0 1 0 {effective_echo_spacing}\n")
        f.write(f"0 -1 0 {effective_echo_spacing}\n")
        f.close()
    
    # Write functinoal acqp file
    with open(out_func,'w') as f:
        for i in range(0,num_frames):
            f.write(f"0 1 0 {effective_echo_spacing}\n")
        f.close()
    
    # Obtain absolute file paths
    out_dist: str = os.path.abspath(out_dist)
    out_func: str = os.path.abspath(out_func)
    
    return out_dist, out_func

In [27]:
def generate_index(num_frames: int,
                   out_file: str = 'file.idx'
                  ) -> str:
    """Creates index files for use with FSL's ``eddy``.
    
    Usage example:
        >>> func_idx = generate_index(num_frames=1200, out_file="fmri.idx")
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        out_file: Output file name.
        
    Returns:
        Output file path as string that contains the corresponding frame indices.
    
    Raises:
        TypeError: Exception that is raised when the input for ``num_frames`` is not an ``int``.
    """
    if type(num_frames) != int:
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer.")
    return np.savetxt(out_file, np.ones((1, num_frames)).T, fmt="%i")

In [6]:
generate_bvals(400)

'file.bval'

In [7]:
generate_bvecs(400)

'file.bvec'

In [23]:
generate_index(400)

In [28]:
generate_acq_params(0.05,400)

('file_distortion_correction.acqp', 'file_functional.acqp')

In [215]:
# slices = 45
slices = 15

In [216]:
# mb_factor = 9
mb_factor = 3

In [217]:
locs = slices//mb_factor
locs

5

In [218]:
step = int(np.round(np.sqrt(locs)))
step
# step = 1

2

In [219]:
# This works for interleaved data
n = []
for s in range(step):
    for k in range(s, locs, step):
        if mb_factor != 1:
            a = [k + locs*j for j in range(mb_factor)]
            n.append(a)
        else:
            a = k
            n.append(a)
n = np.array(n)
# n = np.transpose(n)
n

array([[ 0,  5, 10],
       [ 2,  7, 12],
       [ 4,  9, 14],
       [ 1,  6, 11],
       [ 3,  8, 13]])

In [132]:
slices = 45
mb_factor = 9

In [133]:
locs = slices//mb_factor
locs

5

In [139]:
# step = int(np.round(np.sqrt(locs)))
# step
step = 1

In [140]:
# This single shot data
n = []
for s in range(step):
    # print(s)
    for k in range(locs):
        # print(k)
        if mb_factor != 1:
            a = [k + locs*n for n in range(mb_factor)]
            n.append(a)
        else:
            a = k
            n.append(a)
    n = np.array(n)
# n = np.transpose(n)
n

array([[ 0,  5, 10, 15, 20, 25, 30, 35, 40],
       [ 1,  6, 11, 16, 21, 26, 31, 36, 41],
       [ 2,  7, 12, 17, 22, 27, 32, 37, 42],
       [ 3,  8, 13, 18, 23, 28, 33, 38, 43],
       [ 4,  9, 14, 19, 24, 29, 34, 39, 44]])

In [20]:
def generate_slice_order(slices: int,
                         mb_factor: int = 1,
                         mode: str = 'interleaved',
                         out_file: str = 'file.slice.order',
                         return_mat: Optional[bool] = False
                        ) -> Union[str,np.array]:
    """Generates the slice acquisition order file for use with ``eddy's`` slice-to-volume motion correction method.
    
    The file generated consists of an (N/m) x m matrix | N = number of slices in the acquisition direction (assumed to be
    the z-direction), and m = multi-band factor.
    
    NOTE:
        Links for details:
            * Documentation: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/UsersGuide#A--slspec
            * Implementation: https://git.fmrib.ox.ac.uk/seanf/dhcp-neonatal-fmri-pipeline/-/blob/master/dhcp/resources/default_func.slorder
            * Implementation: https://git.fmrib.ox.ac.uk/matteob/dHCP_neo_dMRI_pipeline_release/-/blob/master/slorder.txt
    
    Usage example:
        >>> sls_order = generate_slice_order(slices=44,
        ...                                  mb_factor=6,
        ...                                  mode='single-shot',
        ...                                  out_file='file.slice.order',
        ...                                  return_mat=False)
        ...
        
    Arguments:
        slices: Number of slices in the acquisition direction.
        mb_factor: Multi-band factor.
        mode: Acquisition algorithm/method/scheme used to acquire the data. Valid options include:
            * ``interleaved``: Optimal slice acquisition technique in which non-adjacent slices are acquired (best for diffusion MR images).
            * ``single-shot``: Slice acquisition in which slices are acquired sequentially with an ascending slice order (best for functional MR images).
            * ``default``: Default acquisition order in which slices are acquired with an ascending slice order.
        out_file: Output file name.
        return_mat: Return a numpy multi-dimensional array (matrix) rather than a file.
        
    Returns:
        Output file path as string that contains the slice acquisition order OR a numpy multi-dimensional array if specified.
    
    Raises:
        TypeError: Exception that is raised in the case that either ``slices`` or ``mb_factor`` is not an ``int``.
        ValueError: Exception that is raised in the case that neither ``interleaved``, ``single-shot``, or ``default`` is passed as an argument for ``mode``.
    """
    # Check input types
    if (type(slices) != int) or (type(mb_factor) != int):
        raise TypeError(f"Input option slices: {slices} or mb_factor: {mb_factor} is not an integer.")
    
    # Locations (in the slices) divided by Multi-Band Factor
    locs: int = slices//mb_factor
    
    if mode == 'interleaved':
        step: int = int(np.round(np.sqrt(locs)))
    elif mode == 'default':
        step: int = 2
    elif mode == 'single-shot':
        step: int = 1
    else:
        raise ValueError(f"Option specified for mode: {mode} does not exist.")
    
    # Iterate through each MB acquisition to get slice ordering
    n: List[int] = []
    
    for s in range(step):
        for k in range(s, locs, step):
            if mb_factor != 1:
                a: List[int] = [ k + locs*j for j in range(mb_factor) ]
                n.append(a)
            else:
                a: int = k
                n.append(a)
    
    if return_mat:
        return np.array(n)
    
    slice_order: np.arrary = np.array(n)
    np.savetxt(out_file,
               slice_order, 
               fmt="%i")
    return out_file

In [4]:
k = 5

In [5]:
type(k).as_integer_ratio()

int

(100, 1)

In [35]:
n = np.array([[0,1,2,3,4]]).T
# n = np.transpose(n).T
n

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [36]:
n.shape

(5, 1)

In [35]:
mb_factor = 4

In [36]:
slices = 44

In [47]:
a = []
for i in range(0,mb_factor):
    # print(i)
    n = []
    for j in range(i,slices,mb_factor):
        n.append(j)
    n = np.array([n])
    a.append(n)
# a = tuple(a)
# a = np.concatenate(a)
a

[array([[ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36, 40]]),
 array([[ 1,  5,  9, 13, 17, 21, 25, 29, 33, 37, 41]]),
 array([[ 2,  6, 10, 14, 18, 22, 26, 30, 34, 38, 42]]),
 array([[ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39, 43]])]

In [ ]:
for i in range(0,)

In [45]:
# Looks cool - but was the incorrect method - this always assumed that the MB factor was 9.
# 
# 
# def generate_slice_order(slices: int,
#                          mb_factor: int = 1,
#                          out_file: str = 'file.slice.order',
#                          return_mat: Optional[bool] = False
#                         ) -> Union[str,np.array]:
#     """working doc-string
    
#     link for details: https://git.fmrib.ox.ac.uk/seanf/dhcp-neonatal-fmri-pipeline/-/blob/master/dhcp/resources/default_func.slorder
#     """
#     try:
#         slice_order: List[int] = []
#         for i in range(0,mb_factor):
#             n: List[int] = []
#             for j in range(i,slices+1,mb_factor):
#                 n.append(j)
#             n: np.array = np.array([n])
#             slice_order.append(n)
        
#         if return_mat:
#             return np.concatenate(slice_order)
        
#         slice_order: np.arrary = np.concatenate(slice_order)
#         np.savetxt(out_file,
#                    slice_order, 
#                    fmt="%i")
#         return out_file
#     except ValueError:
#         return generate_slice_order(slices=slices-1,
#                                     mb_factor=mb_factor,
#                                     return_mat=return_mat)

In [35]:
slice_order_mat: np.array = generate_slice_order(slices=45,mb_factor=9,mode='single-shot',return_mat=True)
slice_order_mat

array([[ 0,  5, 10, 15, 20, 25, 30, 35, 40],
       [ 1,  6, 11, 16, 21, 26, 31, 36, 41],
       [ 2,  7, 12, 17, 22, 27, 32, 37, 42],
       [ 3,  8, 13, 18, 23, 28, 33, 38, 43],
       [ 4,  9, 14, 19, 24, 29, 34, 39, 44]])

In [36]:
# Define the mp_order (Movement parameter order)
mp_order: int = slice_order_mat.shape[0]-1
mp_order

4

In [2]:
import numpy as np
import nibabel as nib

In [3]:
nii_file = "b0s.nii.gz"

In [4]:
img = nib.load(nii_file)

In [10]:
img.header['dim'][3]

45

In [11]:
2%2

0

In [12]:
def _is_even(num: int) -> bool:
    """working doc-string
    """
    if (num%2 == 0):
        return True
    else:
        return False

In [14]:
_is_even(44)

True

# Test ``utils`` implementation details

In [1]:
import utils

In [2]:
f = utils.NiiFile('file.test.nii.gz')

In [3]:
f

file.test.nii.gz

In [4]:
f.ext

'.nii.gz'